# `Calibration`: 



## what is it?
High frequency measurements require the measurement system
to be calibrated

To calibrate is to:
    
 *Remove the unwanted electrical effects of 
 intermediary circuitry between the internals
 of the VNA and the test ports.*
 
<center><img src="figures/vna.jpg" width="400">

## How many types are there?
Most common calibration algorithms can be categorized as
follows
*  One-port (OSL, SDDL, PHN,...)
* Two-port   (TRL, LRM, TRM, ...)
    * 16-term
    * 12-term
    * 8-term  
* N-port

Other things exist;

* Multi-line TRL (Multi-cal)
* iterative solutions (Statisti-Cal)
* multi-frequency

# One Port

<img src="figures/slottedLine.jpg" width="600">

<img src="figures/Microstrip-Directional-Coupler.jpg" width="300"  align="left">
<img src="figures/oldDays.jpg" width="300">
    

<img src="figures/vnaBlockDiagramOnePortRotated.svg" width="600"><img src="figures/Microstrip-Directional-Coupler.jpg" width="300"  align="left">


<img src="figures/vnaBlockDiagramOnePortWithErrorBox.svg" width="600">


<img src="figures/vnaBlockDiagramOnePortWithErrorBoxWaves.svg" width="600">


<img src="figures/sfg3term.svg" width="600">

In [ ]:
from skrf import air50 as m 
m.npoints=401
N = m.impedance_mismatch(1,2)**m.line(400)**m.impedance_mismatch(3,2)**m.line(1000)**m.attenuator(-10)
ideals = {'open':m.open(name='open'),
          'short':m.short(name='short'),
          'load':m.load(rf.db_2_mag(-55),name='load')}

measured= {k:N**ideals[k] for k in ideals}


rf.write_dict_of_networks(ideals, dir = 'data/calibration/ideals/')
rf.write_dict_of_networks(measured, dir = 'data/calibration/measured/')


In [ ]:

import skrf as rf
from skrf.calibration import OnePort

## created necessary data for Calibration class
d= 'data/calibration/'
# a list of Network types, holding 'ideal' responses
my_ideals = [\
        rf.Network(d+'ideals/short.s1p'),
        rf.Network(d+'ideals/open.s1p'),
        rf.Network(d+'ideals/load.s1p'),
        ]

# a list of Network types, holding 'measured' responses
my_measured = [\
        rf.Network(d+'measured/short.s1p'),
        rf.Network(d+'measured/open.s1p'),
        rf.Network(d+'measured/load.s1p'),
        ]

## create a Calibration instance
cal = rf.OnePort(\
        ideals = my_ideals,
        measured = my_measured,
        )


## run, and apply calibration to a DUT

# run calibration algorithm
cal.run()

# apply it to a dut
dut = rf.Network('my_dut.s1p')
dut_caled = cal.apply_cal(dut)
dut_caled.name =  dut.name + ' corrected'

# plot results
dut_caled.plot_s_db()
# save results
dut_caled.write_touchstone()